
# **Install libraries**

In [7]:
!pip install datasets tqdm pandas

In [8]:
!pip install sentencepiece

In [9]:
!pip install transformers

In [10]:
!pip install wandb

In [11]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [12]:
# Check we have a GPU and check the memory size of the GPU
!nvidia-smi

Sun Sep 10 12:00:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

# **Import packages**

In [13]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Set a seed**

In [14]:
import random
import numpy as np
import torch
import datasets

In [15]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)



```
# This is formatted as code
```

# ***C4-200M dataset***

In [16]:
pd.set_option('display.max_colwidth', None)

In [17]:
df = pd.read_csv('/kaggle/input/bec-dataset/train_data.csv')
df.shape

(8032, 6)

In [18]:
df.head()

,Video Title,Genre,Comment,Error,Category,Correct Form
0,"ওবায়দুল কাদের বললেন, ‘খেলা হবে’; আর রুমিন ফারহানা বললেন, ‘আসেন খেলি’ ! | Quader | Rumeen Farhana",Politics,কাদের কি খেলব কাদের তো খেলতেই পারে না,1,Grammatical,কাদের কি খেলবে কাদের তো খেলতেই পারে না
1,পুলিশের গাড়ির ওপর চড়াও বিএনপির কর্মীরা | BNP | Channel 24,Politics,এসব করে আরো কোন ঠাসা হবে,1,Spelling,এসব করে আরো কোণঠাসা হবে
2,Ayub Bachchu | Ek Akash Tara | আইয়ুব বাচ্চু | এক আকাশ তারা | Official Music Video,Entertainment,যুগ যুগ ধরে আমাদের মনে গেথে থাকবে এ গান,0,NaN,যুগ যুগ ধরে আমাদের মনে গেথে থাকবে এ গান
3,যে প্রেম কাহিনী কোন বাধা মানেনি | BBC Bangla,Miscellaneous,অাচছা অাপু এলাজী থাকলে টিকা নেওয়া জাবেনা,1,Spelling,আচ্ছা আপু এলার্জী থাকলে টিকা নেওয়া যাবেনা
4,তুরস্কের চেয়ে ভয়াবহ ভূমিকম্পের ঝুঁকিতে বাংলাদেশ | BBC Bangla,News,হে আল্লাহ এই জালিমদের থেকে আমাদের সন্তান সন্তদের কে আপনি হেফাজত করেন,0,NaN,হে আল্লাহ এই জালিমদের থেকে আমাদের সন্তান সন্তদের কে আপনি হেফাজত করেন


In [19]:
from transformers import (
    T5ForConditionalGeneration, AutoTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [20]:
model_name = 'csebuetnlp/banglat5'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [21]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [22]:
train_df = pd.read_csv('/kaggle/input/bec-dataset/train_data.csv')
test_df = pd.read_csv('/kaggle/input/bec-dataset/test_data.csv')

In [23]:
!pip install git+https://github.com/csebuetnlp/normalizer


  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-rsp26405
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-rsp26405
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6862 sha256=ca6f959c876d150ed80f9b21a85991f2d9691944b611342de4b9cf10b1b4d769
  Stored in directory: /tmp/pip-ephem-wheel-cache-p9xnrd9z/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186458 sha256=37eaab76586dbfde68a81f05385c17b4ead

In [24]:
from normalizer import normalize
train_df['Comment'] = train_df['Comment'].apply(normalize)
# train_df[['Comment','Correct Form']] = train_df[['Comment','Correct Form']].apply(normalize)
# test_df[['Comment','Correct Form']] = test_df[['Comment','Correct Form']].apply(normalize)

In [25]:
train_df['Correct Form'] = train_df['Correct Form'].apply(normalize)
test_df['Comment'] = test_df['Comment'].apply(normalize)
test_df['Correct Form'] = test_df['Correct Form'].apply(normalize)

In [26]:
train_df.shape, test_df.shape

((8032, 6), (2010, 6))

In [89]:
train_df['input_token_len'] = train_df['Comment'].apply(calc_token_len)

In [27]:
test_df['input_token_len'] = test_df['Comment'].apply(calc_token_len)

In [28]:
test_df.head()

,Video Title,Genre,Comment,Error,Category,Correct Form,input_token_len
0,নেত্রীর কথা শুনলে এখন হাজার হাজার শ্রমিক রাস্তায় নেমে যাবে' | Sheikh Sharhan Naser Tonmoy,Politics,আওয়ামী লীগের এতো লোক তাহলে কেন এত ভয় পায়,0,NaN,আওয়ামী লীগের এতো লোক তাহলে কেন এত ভয় পায়,10
1,RedMagic 8 Pro দেখে আমি তো অবাক 😮,Miscellaneous,প্রাইস টা বললে কি হতো রে ইমন,1,Code Switching,দাম টা বললে কি হতো রে ইমন,8
2,অস্থির বাঙালি Part 35😂 osthir bengali | funny video | funny facts | facts bangla,Entertainment,এত সুন্দর হাসি ভালো লাগল,1,Spelling,এত সুন্দর হাসি ভালো লাগলো,6
3,দেশে প্রথমবারের মতো চ্যানেল 24-এর পর্দায় সংবাদ পাঠ করলেন এআই 'অপরাজিতা' | AI Presenter | Channel 24,News,সময় চলে এসেছে আপনাদেরকে বিদায় জানাবার,0,NaN,সময় চলে এসেছে আপনাদেরকে বিদায় জানাবার,9
4,"মীনা, রাজু, মিঠুর মিমিক্রি করে তাক লাগিয়ে দিয়েছেন অথৈ | Oitijya Authoi Roy | Voice Artist | Somoy TV",Entertainment,চুল একটু বড় হলে ভালো হত না,1,Spelling,চুল একটু বড় হলে ভালো হতো না,8


In [29]:
test_df['input_token_len'].describe()

count    2010.000000
mean       11.428358
std         6.981458
min         4.000000
25%         7.000000
50%        10.000000
75%        13.000000
max       111.000000
Name: input_token_len, dtype: float64

### We will use a token length of 64 since it will cover the vast majority of examples

In [30]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [31]:
test_dataset

Dataset({
    features: ['Video Title', 'Genre', 'Comment', 'Error', 'Category', 'Correct Form', 'input_token_len'],
    num_rows: 2010
})

### Load the Dataset

In [32]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['Comment'], example['Correct Form']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [33]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 4
attention_mask 4
labels 4
{'input_ids': [179, 1158, 3409, 1], 'attention_mask': [1, 1, 1, 1], 'labels': [179, 1158, 3409, 1]}


### Define Evaluator

In [34]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f1fd739189d6f5a58f18160f5dc3459130e5ee913fc71b34bb9f800896ededb7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [35]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

### Train Model

In [36]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [37]:
!pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
!pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [39]:
# defining training related arguments
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=100,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")

In [40]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=False)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

In [42]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,7.835800,1.879248,0.000000,0.000000,0.000000,0.000000,8.351700
1000,2.255600,0.900580,0.000000,0.000000,0.000000,0.000000,9.825900
1500,1.254600,0.483554,0.000000,0.000000,0.000000,0.000000,10.444300
2000,0.928600,0.427365,0.000000,0.000000,0.000000,0.000000,10.454700
2500,0.795000,0.408037,0.000000,0.000000,0.000000,0.000000,10.453200
3000,0.710500,0.398556,0.000000,0.000000,0.000000,0.000000,10.446800
3500,0.667100,0.389527,0.000000,0.000000,0.000000,0.000000,10.428400
4000,0.639400,0.386778,0.000000,0.000000,0.000000,0.000000,10.429900


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=4100, training_loss=1.8552963666218083, metrics={'train_runtime': 13587.8711, 'train_samples_per_second': 59.112, 'train_steps_per_second': 0.302, 'total_flos': 3.449529358958592e+16, 'train_loss': 1.8552963666218083, 'epoch': 98.01})

In [43]:
trainer.save_model('bangla_gec_model')

In [44]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [45]:
!zip -r 'bangla_gec_model.zip' 'bangla_gec_model'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: bangla_gec_model/ (stored 0%)
  adding: bangla_gec_model/special_tokens_map.json (deflated 86%)
  adding: bangla_gec_model/tokenizer.json (deflated 76%)
  adding: bangla_gec_model/generation_config.json (deflated 29%)
  adding: bangla_gec_model/training_args.bin (deflated 49%)
  adding: bangla_gec_model/pytorch_model.bin (deflated 10%)
  adding: bangla_gec_model/tokenizer_config.json (deflated 83%)
  adding: bangla_gec_model/spiece.model (deflated 60%)
  adding: bangla_gec_model/config.json (deflated 48%)


In [46]:
!mv bangla_gec_model.zip /model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


I have uploaded this model to HuggingFace Model Zoo and we can run inference using it

## Testing

In [47]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = 'bangla_gec_model'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences,input_len):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=input_len, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=input_len,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [48]:
references,predictions = [],[]
test_d = test_df[test_df['Error']==1]
test_d
# for sentence in test_sentences:
#   print(f"input sentence:",sentence)
#   references.append(sentence)
#   output_sentence = correct_grammar(sentence,num_return_sequences=2,)
#   print("output sentence: ",output_sentence)
#   predictions.append(output_sentence[0])

,Video Title,Genre,Comment,Error,Category,Correct Form,input_token_len
1,RedMagic 8 Pro দেখে আমি তো অবাক 😮,Miscellaneous,প্রাইস টা বললে কি হতো রে ইমন,1,Code Switching,দাম টা বললে কি হতো রে ইমন,8
2,অস্থির বাঙালি Part 35😂 osthir bengali | funny video | funny facts | facts bangla,Entertainment,এত সুন্দর হাসি ভালো লাগল,1,Spelling,এত সুন্দর হাসি ভালো লাগলো,6
4,"মীনা, রাজু, মিঠুর মিমিক্রি করে তাক লাগিয়ে দিয়েছেন অথৈ | Oitijya Authoi Roy | Voice Artist | Somoy TV",Entertainment,চুল একটু বড় হলে ভালো হত না,1,Spelling,চুল একটু বড় হলে ভালো হতো না,8
5,ঘোড়া কীভাবে সাপের কামড় থেকে মানুষকে বাঁচায়? | Why horse is used for antivenom? | Jamuna TV,Miscellaneous,সমস্যা হলো সকল সরকারি হসপিটালে অন্টিভেনোম না পাওয়া,1,Spelling,সমস্যা হলো সকল সরকারি হসপিটালে এন্টিভ্যানম না পাওয়া,12
6,খেলা হবে' রাজনীতির মাঠে আবারও আলোচনায় কেন? | BBC Bangla,Politics,চেয়ার খেলা শুরু করে খেলার উদ্বোধন করলেন প্রধানমন্ত্রী লোকেরা মোজা পাইছি কাদের কাউয়া,1,Spelling,চেয়ার খেলা শুরু করে খেলার উদ্বোধন করলেন প্রধানমন্ত্রী লোকেরা মজা পেয়েছে কাদের কাউয়া,16
...,...,...,...,...,...,...,...
1996,"Liver Cirrhosis: লিভার সিরোসিস রোগের কারণ, লক্ষণ ও চিকিৎসা কী? | BBC Bangla",Miscellaneous,এই রুগটা কি বন্স গতো জানা বেন,1,Spelling,এই রোগটা কি বংশগতো জানাবেন,12
2003,সিরাহ ৭ – অ্যাবিসিনিয়া | Bangla Seerah,Miscellaneous,মাশাআল্লাহ এই লোকটি আমার প্রিয় মানুসের মধ্যে একজন,1,Spelling,মাশাআল্লাহ এই লোকটি আমার প্রিয় মানুষের মধ্যে একজন,12
2004,তুরস্কের চেয়ে ভয়াবহ ভূমিকম্পের ঝুঁকিতে বাংলাদেশ | BBC Bangla,News,আল্লাহ তুমি বনটিকে জান্নাতে তার সামী সাথে তাকতে দিয় আমিন,1,Multiple Errors,আল্লাহ তুমি বোনটিকে জান্নাতে তার স্বামীর সাথে থাকতে দিয়ো আমিন,15
2007,খালেদা জিয়া’র ব্যাপারে আর কত করবো ? প্রশ্ন প্রধানমন্ত্রীর | Sheikh Hasina | Khaleda Zia,Politics,আর কত করব তাও তো মরলনা,1,Spelling,আর কত করবো তাও তো মরলো না,9


In [128]:
test_d_sentence = test_d['Comment'].tolist()
test_d_len = test_d['input_token_len'].tolist()
test_d_ground = test_d['Correct Form'].tolist()

In [129]:
import nltk
nltk.download('punkt')
from nltk.util import ngrams
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [130]:
def calculate(reference,hypothesis):
    # Tokenize sentences into words
    reference_tokens = nltk.word_tokenize(reference)
    hypothesis_tokens = nltk.word_tokenize(hypothesis)

    # Create n-grams for reference and hypothesis
    reference_1grams = list(ngrams(reference_tokens, 1))
    hypothesis_1grams = list(ngrams(hypothesis_tokens, 1))
    reference_2grams = list(ngrams(reference_tokens, 2))
    hypothesis_2grams = list(ngrams(hypothesis_tokens, 2))

    # Calculate ROUGE scores
    rouge1_precision = len(set(reference_1grams).intersection(hypothesis_1grams)) / len(reference_1grams)
    rouge1_recall = len(set(reference_1grams).intersection(hypothesis_1grams)) / len(hypothesis_1grams)
    rouge2_precision = len(set(reference_2grams).intersection(hypothesis_2grams)) / len(reference_2grams)
    rouge2_recall = len(set(reference_2grams).intersection(hypothesis_2grams)) / len(hypothesis_2grams)

    # Calculate ROUGE-L using NLTK's sentence_bleu function
    smooth = SmoothingFunction().method4
    rougeL = sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smooth)
    d = {
        "rouge1_precision":rouge1_precision,
        "rouge1_recall":rouge1_recall,
        "rouge2_precision":rouge2_precision,
        "rouge2_recall":rouge2_recall,
        "rouge_l":rougeL
    }
    return d

In [134]:
import nltk
from nltk.util import ngrams

# Function to calculate ROUGE-1, ROUGE-2, and ROUGE-L scores for a pair of texts
def calculate_rouge_scores(reference_tokens, system_tokens):
    def lcs(X, Y):
        m, n = len(X), len(Y)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if X[i - 1] == Y[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + 1
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

        return dp[m][n]

    # Calculate ROUGE-1 (unigram) scores
    reference_unigrams = set(reference_tokens)
    system_unigrams = set(system_tokens)
    overlap_rouge1 = len(reference_unigrams.intersection(system_unigrams))
    precision_rouge1 = overlap_rouge1 / len(system_unigrams)
    recall_rouge1 = overlap_rouge1 / len(reference_unigrams)
    r1_t = 1 if precision_rouge1 + recall_rouge1 == 0 else 0
    f1_rouge1 = 2 * (precision_rouge1 * recall_rouge1) / (precision_rouge1 + recall_rouge1 + r1_t)

    # Calculate ROUGE-2 (bigram) scores
    reference_bigrams = set(ngrams(reference_tokens, 2))
    system_bigrams = set(ngrams(system_tokens, 2))
    overlap_rouge2 = len(reference_bigrams.intersection(system_bigrams))
    precision_rouge2 = overlap_rouge2 / len(system_bigrams)
    recall_rouge2 = overlap_rouge2 / len(reference_bigrams)
    r2_t = 1 if precision_rouge2 + recall_rouge2 == 0 else 1
    f1_rouge2 = 2 * (precision_rouge2 * recall_rouge2) / (precision_rouge2 + recall_rouge2 + r2_t)

    # Calculate ROUGE-L scores
    lcs_length = lcs(reference_tokens, system_tokens)
    precision_rougeL = lcs_length / len(system_tokens)
    recall_rougeL = lcs_length / len(reference_tokens)
    rL_t = 1 if precision_rougeL + recall_rougeL == 0 else 0
    f1_rougeL = 2 * (precision_rougeL * recall_rougeL) / (precision_rougeL + recall_rougeL + rL_t)

    return {
        'ROUGE-1 Precision': precision_rouge1,
        'ROUGE-1 Recall': recall_rouge1,
        'ROUGE-1 F1': f1_rouge1,
        'ROUGE-2 Precision': precision_rouge2,
        'ROUGE-2 Recall': recall_rouge2,
        'ROUGE-2 F1': f1_rouge2,
        'ROUGE-L Precision': precision_rougeL,
        'ROUGE-L Recall': recall_rougeL,
        'ROUGE-L F1': f1_rougeL,
    }

# Function to calculate the average of ROUGE scores for an array of text pairs
def calculate_average_rouge_scores(reference_texts, system_texts):
    total_scores = {
        'ROUGE-1 Precision': 0,
        'ROUGE-1 Recall': 0,
        'ROUGE-1 F1': 0,
        'ROUGE-2 Precision': 0,
        'ROUGE-2 Recall': 0,
        'ROUGE-2 F1': 0,
        'ROUGE-L Precision': 0,
        'ROUGE-L Recall': 0,
        'ROUGE-L F1': 0,
    }

    num_pairs = len(reference_texts)

    for i in range(num_pairs):
        reference_text = reference_texts[i]
        system_text = system_texts[i]

        reference_tokens = nltk.word_tokenize(reference_text)
        system_tokens = nltk.word_tokenize(system_text)

        scores = calculate_rouge_scores(reference_tokens, system_tokens)

        for key, value in scores.items():
            total_scores[key] += value

    # Calculate the average scores
    average_scores = {key: value / num_pairs for key, value in total_scores.items()}
    
    return average_scores

# Example usage with an array of reference and system texts
reference_texts = test_d_ground
system_texts = [correct_grammar(test_d_sentence[i],num_return_sequences=2,input_len=test_d_len[i])[0] for i in range(len(test_d_sentence)) ]

average_scores = calculate_average_rouge_scores(reference_texts, system_texts)
print("Average ROUGE Scores:")
for key, value in average_scores.items():
    print(key + ": {:.4f}".format(value))

Average ROUGE Scores:
ROUGE-1 Precision: 0.8477
ROUGE-1 Recall: 0.8460
ROUGE-1 F1: 0.8461
ROUGE-2 Precision: 0.7306
ROUGE-2 Recall: 0.7283
ROUGE-2 F1: 0.4430
ROUGE-L Precision: 0.8475
ROUGE-L Recall: 0.8457
ROUGE-L F1: 0.8459


In [101]:
# random data

r = train_df['Comment'].tolist()[:20] + test_df['Comment'].tolist()[:20]
r_ln = train_df['input_token_len'].tolist()[:20] + test_df['input_token_len'].tolist()[:20]
reference_texts = r
for i in range(len(r)):
    print(f'[input:   ] {r[i]} and [output:   ] {correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]}')

[input:   ] কাদের কি খেলব কাদের তো খেলতেই পারে না and [output:   ] কাদের কি খেলব কাদের তো খেলতেই পারে না
[input:   ] এসব করে আরো কোন ঠাসা হবে and [output:   ] এসব করে আরো কোন ঠাসা হবে
[input:   ] যুগ যুগ ধরে আমাদের মনে গেথে থাকবে এ গান and [output:   ] যুগ যুগ ধরে আমাদের মনে গেথে থাকবে এ গান
[input:   ] আচছা আপু এলাজী থাকলে টিকা নেওয়া জাবেনা and [output:   ] আচছা আপু এলাজী থাকলে টিকা নেওয়া যাবেনা
[input:   ] হে আল্লাহ এই জালিমদের থেকে আমাদের সন্তান সন্তদের কে আপনি হেফাজত করেন and [output:   ] হে আল্লাহ এই জালিমদের থেকে আমাদের সন্তান সন্তদের কে আপনি হেফাজত করেন
[input:   ] আমার মতে মাহমুদুল্লাহ রিয়াদ বাংলাদেশের সেরা খেলোয়াড় and [output:   ] আমার মতে মাহমুদুল্লাহ রিয়াদ বাংলাদেশের সেরা খেলোয়াড়
[input:   ] কথা গুলো শুনে কান্না করে দিয়েছি কারণ আমি ও এতীম এই পৃথিবীতে এতীম কোন দাম নেই পৃথিবীর মানুষ গুলো বড়ো স্বার্থ পর and [output:   ] কথা গুলো শুনে কান্না করে দিয়েছি কারণ আমি ও এতীম এই পৃথিবীতে এতীম কোন দাম নেই পৃথিবীর মানুষ গুলো বড়ো স্বার্থ পর
[input:   ] ইয়া আল্লাহ্ তুমিই রক্ষার

In [103]:
test_df['Error'].value_counts()

Error
0    1167
1     843
Name: count, dtype: int64

In [107]:
# error data

import time

r = (test_df[test_df['Error']==1])['Comment'].tolist()
r_ln = (test_df[test_df['Error']==1])['input_token_len'].tolist()
reference_texts = r
start_time = time.time()
for i in range(len(r)):
    x = r[i]
    y = correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]
#     print(f'[input:   ] {r[i]} and [output:   ] {correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]}')
end_time = time.time()
num_iterations = len(r)
average_inference_time = (end_time - start_time) / num_iterations
print(f"Total Inference Time: {end_time-start_time}")
print(f"Average Inference Time: {average_inference_time:.4f} seconds")

Total Inference Time: 220.37577748298645
Average Inference Time: 0.2614 seconds


In [136]:
r_gn = (test_df[test_df['Error']==1])['Correct Form'].tolist()

In [138]:
predicted_score,predicted_sentences = [],[]
for i in range(len(r_gn)):
    x = r_gn[i]
    y = correct_grammar(r[i],num_return_sequences=2,input_len=r_ln[i])[0]
    predicted_sentences.append(y)
    rouge_score_v = calculate_rouge_scores(x, y)
    predicted_score.append(rouge_score_v['ROUGE-L F1'])
# test_df['ROUGE-L F1'] = predicted_sentence

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [139]:
e_df = test_df[test_df['Error']==1]
e_df['ROUGE-L F1'] = predicted_score
e_df['Predicted Form'] = predicted_sentences

/tmp/ipykernel_29/2734637383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['ROUGE-L F1'] = predicted_score
/tmp/ipykernel_29/2734637383.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df['Predicted Form'] = predicted_sentences


In [140]:
e_df.sort_values(by='ROUGE-L F1', inplace=True,ascending=False)

/tmp/ipykernel_29/3169983084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_df.sort_values(by='ROUGE-L F1', inplace=True,ascending=False)


In [149]:
e_df.head(20)[1:]

,Video Title,Genre,Comment,Error,Category,Correct Form,input_token_len,ROUGE-L F1,Predicted Form
377,"Coffee Houser sei addata Video, Debashis Sengupta_",Entertainment,জীবন ফুরিয়ে যাবা তবে গান গুলো কখনোই ফুরিয়ে যাবে না যতদিন বাংলা ভাষা বেঁচে থাকবে,1,Grammatical,জীবন ফুরিয়ে যাবে তবে গান গুলো কখনোই ফুরিয়ে যাবে না যতদিন বাংলা ভাষা বেঁচে থাকবে,17,1.0,জীবন ফুরিয়ে যাবে তবে গান গুলো কখনোই ফুরিয়ে যাবে না যতদিন বাংলা ভাষা বেঁচে থাকবে
808,Khulna Travel Vlog | খুলনা ভ্রমন গল্প,Miscellaneous,আমাদের খুলনা এত সুন্দর করে ভিডিও করে আমাদের উপহার দেয়ার জন্য থ্যাংক ইউ আপনাকে ভাইয়া,1,Code Switching,আমাদের খুলনা এত সুন্দর করে ভিডিও করে আমাদের উপহার দেয়ার জন্য ধন্যবাদ আপনাকে ভাইয়া,16,1.0,আমাদের খুলনা এত সুন্দর করে ভিডিও করে আমাদের উপহার দেয়ার জন্য ধন্যবাদ আপনাকে ভাইয়া
807,আদানির সাথে বিদ্যুৎ চুক্তি কতটা ক্ষতিকর ?,Politics,থ্যাংক ইউ ভাই এমন বাস্তব তথ্যবহুল এবং সময়োপযোগি ভিডিও বানানোর জন্য,1,Code Switching,ধন্যবাদ ভাই এমন বাস্তব তথ্যবহুল এবং সময়োপযোগি ভিডিও বানানোর জন্য,16,1.0,ধন্যবাদ ভাই এমন বাস্তব তথ্যবহুল এবং সময়োপযোগি ভিডিও বানানোর জন্য
802,মহর্ষি ভৃগু কেন বিষ্ণুর বুকে পদাঘাত করেছিলেন? Why Rishi Bhrigu Kicked on The Chest of Lord Vishnu?,Miscellaneous,শুনলাম বেশ ভালো লাগলো থ্যাংক ইউ,1,Code Switching,শুনলাম বেশ ভালো লাগলো ধন্যবাদ,7,1.0,শুনলাম বেশ ভালো লাগলো ধন্যবাদ
1639,"পূজা মণ্ডপে কোরআন: ঢাকা ও নোয়াখালীতে জুমার নামাজের পর বিক্ষোভ, সংঘর্ষ ও হামলা | BBC Bangla",News,আল্লাহ্ সবাইকে রক্ষা করবাে,1,Grammatical,আল্লাহ্ সবাইকে রক্ষা করবে,6,1.0,আল্লাহ্ সবাইকে রক্ষা করবে
793,Your Friendly Pet Robot - EMO | ATC,Miscellaneous,ভাই কোই পাওয়া যাবে দাম কত টাকা,1,Spelling,ভাই কই পাওয়া যাবে দাম কত টাকা,9,1.0,ভাই কই পাওয়া যাবে দাম কত টাকা
782,শাকিব খান এবং অপু বিশ্বাসকে নিয়ে বুবলির সম্পূর্ণ ইন্টারভিউ,Miscellaneous,কাউকে কষ্ট দিয়ে কেউ কখনো সুখী হতে পাড়ে না,1,Spelling,কাউকে কষ্ট দিয়ে কেউ কখনো সুখী হতে পারে না,10,1.0,কাউকে কষ্ট দিয়ে কেউ কখনো সুখী হতে পারে না
771,"রিয়াদ প্রশ্নে উত্তর হাতড়ালেন পাপন, মাশরাফিকে নিয়ে রাখঢাক ! | Sports | Khelajog | Ekattor TV",Sports,মাহমুদউল্লাহ বাইকে জাতীয় দলে দেকতে চাই,1,Spelling,মাহমুদউল্লাহ বাইকে জাতীয় দলে দেখতে চাই,9,1.0,মাহমুদউল্লাহ বাইকে জাতীয় দলে দেখতে চাই
768,চাপাতি দিয়ে কুপিয়ে যুবলীগ কর্মীকে হ'ত্যা | Jubo League workers | Murder | Rtv News,News,এই দেশে সঠিক বিচার আশা করা যাই না,1,Spelling,এই দেশে সঠিক বিচার আশা করা যায় না,9,1.0,এই দেশে সঠিক বিচার আশা করা যায় না
761,আন্দালিব রহমান পার্থ: শেখ হাসিনার আত্মীয় হয়েও আওয়ামী লীগের কঠোর সমালোচনা কতটা চাপের?,Politics,বাংলা দেশের মানুস জিয়া কে ভালবাসে,1,Spelling,বাংলা দেশের মানুষ জিয়া কে ভালবাসে,8,1.0,বাংলা দেশের মানুষ জিয়া কে ভালবাসে


In [148]:
e_df.tail(20)

,Video Title,Genre,Comment,Error,Category,Correct Form,input_token_len,ROUGE-L F1,Predicted Form
432,Meye | মেয়ে | Ayub Bachchu | Niaz Ahmed Aungshu | AB Kitchen,Entertainment,লিজেন্ড কে হারিয়ে ফেরেছি আমরা,1,Multiple Errors,কিংবদন্তীকে কে হারিয়ে ফেলেছি আমরা,8,0.750000,লিজেন্ড কে হারিয়ে ফিরেছি আমরা
354,ইতিহাসের সবথেকে ভয়ংকর দুর্গ | যেখানে একবার ঢুকলে কেউ আর ফেরত আসতো না | History of Daulatabad Fort,Miscellaneous,এই কেল্লা সত্যিই সিভিল ইঞ্জিনিয়ারিং এর আশ্চর্য ইতিহাস,1,Code Switching,এই কেল্লা সত্যিই নির্মাণ প্রকৌশলী এর আশ্চর্য ইতিহাস,9,0.742857,এই কেল্লা সত্যিই সিভিল ইঞ্জিনিয়ারিং এর আশ্চর্য ইতিহাস
1367,আমি বিশ্বের সবচেয়ে উষ্ণ স্থানে গিয়েছিলাম (৭০.৭°সে.) লুত মরুভূমি,Miscellaneous,অসাধারণ একটি ডকুমেন্টারি চ্যানেল,1,Code Switching,অসাধারণ একটি তথ্যচিত্র চ্যানেল,5,0.741935,অসাধারণ একটি ডকুমেন্টারি চ্যানেল
1493,Xiaomi 13 Lite - এমন ফোনই আমরা চাই !,Miscellaneous,শাওমির ফোন আমারে ফ্রী দিলেও তো আমি ইউজ করবো থার্ড ক্লাস মার্কা,1,Code Switching,শাওমির ফোন আমাকে বিনামূল্যে দিলেও তো আমি ব্যবহার করবো তৃতীয় ক্লাস মার্কা,16,0.740741,শাওমির ফোন আমারে ফ্রী দিলেও তো আমি ইউজ করবো থার্ড ক্লাস মার্কা
335,পশুদের মজার কর্মকান্ড ক্যামেরায় ধরা পড়া | Funny Animals Video 2022 (Part-3) | mayajaal,Entertainment,ডায়লোগ গুলো বেশি বেশি হইছে,1,Spelling,সংলাপ গুলো বেশি বেশি হয়েছে,8,0.740741,ডায়লোগ গুলো বেশি বেশি হইছে
1403,SURONGO | Official Foretaste | Afran Nisho | Tama Mirza | Raihan Rafi | Alpha-i | Chorki,Entertainment,এইটা কোন মুভির ট্রেইলার রইল একটা নাটক,1,Code Switching,এইটা কোন চলচ্চিত্রের খন্ড রইল একটা নাটক,9,0.736842,এইটা কোন মুভির ট্রেইলার রইল একটা নাটক
1595,Nothing Phone 1 First Time Unboxing & Impression 🇧🇩 | ATC,Miscellaneous,মিড বাজেটে ওয়াটারপ্রুফ ফোন কোনটা ভালো হবে,1,Code Switching,মধ্য বাজেটে জলরোধী ফোন কোনটা ভালো হবে,11,0.734177,মিড বাজেটে ওয়াটারপ্রুফ ফোন কোনটা ভালো হবে
394,ঋতু পরিবর্তন এবং মরুভূমি Season change on earth and Desert explained in Bangla Ep 107,Miscellaneous,প্রথম ভিও আমার,1,Code Switching,প্রথম দর্শন আমার,5,0.733333,প্রথম ভিও আমার
1675,[২য় পর্ব] সেরা কয়েকটি দৃষ্টিভ্রম | Top optical and sound illusion bangla|,Miscellaneous,এমন ভিডিও আরো চাই প্লিজ,1,Code Switching,এমন ভিডিও আরো চাই অনুগ্রহ করে,6,0.730769,এমন ভিডিও আরো চাই প্লিজ
342,ছায়াপথ বা গ্যালাক্সি | কি কেন কিভাবে | Galaxy | Ki Keno Kivabe,Miscellaneous,ভাইয়া ব্লকহোল নিয়ে একটি ভিডিও বানাবেন প্লিজ,1,Code Switching,ভাইয়া কৃষ্ণ গহ্বর নিয়ে একটি ভিডিও বানাবেন অনুগ্রহ করে,10,0.720000,ভাইয়া ব্লকহোল নিয়ে একটি ভিডিও বানাবেন প্লিজ


In [76]:
for i in range(7):
    input_sentence = test_d_sentence[i]
    # print('Input sentence is : {}'.format(input_sentence))
    y = correct_grammar(input_sentence,num_return_sequences=2,input_len=test_d_len[i])[0]
    print(calculate(input_sentence,y))

{'rouge1_precision': 1.0, 'rouge1_recall': 1.0, 'rouge2_precision': 1.0, 'rouge2_recall': 1.0, 'rouge_l': 1.0}
{'rouge1_precision': 1.0, 'rouge1_recall': 1.0, 'rouge2_precision': 1.0, 'rouge2_recall': 1.0, 'rouge_l': 1.0}
{'rouge1_precision': 0.8571428571428571, 'rouge1_recall': 0.8571428571428571, 'rouge2_precision': 0.6666666666666666, 'rouge2_recall': 0.6666666666666666, 'rouge_l': 0.6434588841607617}
{'rouge1_precision': 0.875, 'rouge1_recall': 0.875, 'rouge2_precision': 0.7142857142857143, 'rouge2_recall': 0.7142857142857143, 'rouge_l': 0.5946035575013605}
{'rouge1_precision': 1.0, 'rouge1_recall': 1.0, 'rouge2_precision': 1.0, 'rouge2_recall': 1.0, 'rouge_l': 1.0}
{'rouge1_precision': 1.0, 'rouge1_recall': 1.0, 'rouge2_precision': 1.0, 'rouge2_recall': 1.0, 'rouge_l': 1.0}
{'rouge1_precision': 1.0, 'rouge1_recall': 1.0, 'rouge2_precision': 1.0, 'rouge2_recall': 1.0, 'rouge_l': 1.0}


In [58]:
!cd /kaggle/working

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [59]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
bangla_gec_model  wandb  weights


In [60]:
from IPython.display import FileLink
FileLink(r'bangla_gec_model.zip')

/kaggle/working/bangla_gec_model.zip